In [1]:
import h5py
import numpy as np    
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image


######## Very basic access to the dataset - let's see what we are working with! #######
raw_dataset = h5py.File('climatevisions_2019_popular.h5','r+') 
dataset = raw_dataset['tweet_data']
image_directory = 'C:\\Users\\Admin\\Documents\\Dataset_small\\'
cols_to_strip = ['created_at', 'img_name', 'language', 'referenced_tweets', 'text', 'tweet_id']   

data_dict = {}
# Iterate through the keys (assuming each key is a column name)
for key in dataset.keys():
     # Access the data for each column
     column_data = dataset[key][:]
        
     # Store the data in the dictionary with the column name as the key
     data_dict[key] = column_data
     
df = pd.DataFrame(data_dict)
df[cols_to_strip] = df[cols_to_strip].astype('string')
df[cols_to_strip] = df[cols_to_strip].replace(to_replace=r'^b\':?(.*)\'$', value=r'\1', regex=True)


print(df.shape)
df.dtypes

## only keep images here
# drop all columns exepct img_ columns
selected_columns = ['img_name']
df_selected = df.loc[:, selected_columns]
df_selected.head()

(5000, 9)


,img_name
0,id_1153283149360762880_2019-07-22.jpg
1,id_1163744643600637952_2019-08-20.jpg
2,id_1122574040936452097_2019-04-28.jpg
3,id_1188805167958974465_2019-10-28.jpg
4,id_1108042949449969666_2019-03-19.jpg


In [37]:
## Write into .csv file

from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline

from PIL import Image
from ultralytics import YOLO
import cv2
from openpyxl import Workbook

wb = Workbook()
ws = wb.active

image_data = df["img_name"].tolist()

header = ["image_name", "object_detection_results", "description_short", "description_long", "description_comparison"]
moondream_captions = {}
aisak_captions = {}

ws.append(header)

i = 0
count = 0
for img_name in image_data: 
    # Load a model
    model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
    animals = ['bear', 'penguin', 'polar bear'] 
   
    img_name = img_name.replace("\n","")
 
    image = Image.open(image_directory + img_name)
   
    results = model(image)  # predict on an image or image directory
    names = model.names
    #bounding boxes and confidence scores
 
    for result in results:
        detected_objects = []
        detections = result.pred[0] if hasattr(result, 'pred') else None  # Get the detections if available
        result.save()
        #result.show()
        # Display the detected objects
        ids = result.boxes.cls
        for id in ids:
            name = names[int(id)]
            if (name not in detected_objects):
                detected_objects.append(name)
        print(detected_objects)
        
        if ('person' in detected_objects):
            #TODO FACE RECOGNITION
            print('person')
        elif (any(animal in detected_objects for animal in animals)):
            #TODO
            print('animals')
        else:
            #TODO
            print('else')
            print()
            
            
    
    # long description model
    model_id = "vikhyatk/moondream2"
    revision = "2024-04-02"
    model = AutoModelForCausalLM.from_pretrained(
        model_id, trust_remote_code=True, revision=revision
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)
 
    enc_image = model.encode_image(image)
    moonDreamResult = model.answer_question(enc_image, "Describe this image.", tokenizer)
    moondream_captions[img_name] = moonDreamResult
    
    # short description model
    imgToText = pipeline("image-to-text", model="aisak-ai/aisak-visual")
    aisakResult = imgToText(image)
    aisak_captions[img_name] = aisakResult
    
    
    # Depth Anything for Depth Estimation - Foreground/Background
    #pipe = pipeline(task="depth-estimation", model="LiheYoung/depth-anything-large-hf")
    #depth = pipe(image)["depth"]
    #cv2.imshow('Depth Anything', depth)
    object_string = ''
    for index, object in enumerate(detected_objects):
        if (index < len(detected_objects)):
            object_string += object + ',' + ' '
        else:
            object_string += object

    i+=1
    if (i == 15):
        break
    
    mapEntry = aisakResult[0]
    firstValueShort = next(iter(mapEntry.values()))

    data_row = [img_name, object_string, moonDreamResult, firstValueShort, '']
    ws.append(data_row)
        
    count += 1
    detected_objects.clear




0: 640x384 (no detections), 57.6ms
Speed: 1.0ms preprocess, 57.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
[]
else



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



0: 416x640 (no detections), 398.9ms
Speed: 1.0ms preprocess, 398.9ms inference, 0.0ms postprocess per image at shape (1, 3, 416, 640)
[]
else



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



0: 480x640 1 bird, 428.4ms
Speed: 3.5ms preprocess, 428.4ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)
['bird']
else



...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



0: 640x384 1 person, 404.3ms
Speed: 2.0ms preprocess, 404.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)
['person']
person


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.



0: 448x640 7 persons, 3 airplanes, 8 ties, 445.6ms
Speed: 2.0ms preprocess, 445.6ms inference, 1.5ms postprocess per image at shape (1, 3, 448, 640)
['person', 'tie', 'airplane']
person


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
import json
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import pairwise


## Comparison of the three captions ###
with open("gemini-captions.json", "r") as file:
    gemini_captions = json.load(file)

i = 0
for img_name in image_data:
    img_name = img_name.replace("\n","")
    image = Image.open(image_directory + img_name)
    gemini_caption = gemini_captions[img_name]
    moondream_caption = moondream_captions[img_name]
    aisak_caption = aisak_captions[img_name]
    aisak_caption = (aisak_caption[0])['generated_text']
    #print(gemini_caption)
    #print(moondream_caption)
    #print(aisak_caption)
    
    
    image.show()
    i += 1
    sbert = AutoModel.from_pretrained("Voicelab/sbert-base-cased-pl")
    tokenizer = AutoTokenizer.from_pretrained("Voicelab/sbert-base-cased-pl")
    tokens = tokenizer([gemini_caption, moondream_caption, aisak_caption], padding=True, truncation=True, return_tensors='pt')
    x = sbert(tokens["input_ids"], tokens["attention_mask"]).pooler_output
    similarity_matrix = pairwise.cosine_similarity(x.detach().numpy())
    #print("Similarities: " + str(similarity_matrix[0,1]) + " " + similarity_matrix[0,2] + " " + similarity_matrix[1,2])
    avg_sim = (similarity_matrix[0,1] + similarity_matrix[0,2] + similarity_matrix[1,2]) / 3
    print("Average: " , avg_sim)
    if (i == 15): 
        break
    
    
    

### Confidence   ###




Average:  0.873517910639445
Average:  0.8720777829488119
Average:  0.903128465016683
Average:  0.9182259241739908
Average:  0.9083538850148519
Average:  0.8902325630187988
Average:  0.9210977554321289
Average:  0.8911571502685547
Average:  0.9032729466756185
Average:  0.9049077828725179
Average:  0.8510869344075521
Average:  0.9078140258789062
Average:  0.9079781373341879
Average:  0.8865758577982584
Average:  0.8991026878356934
